In [1]:
import requests
from urllib.parse import urlencode, quote

In [2]:
class FortiAPIClient():

    def __init__(self, host, session=None, protocol='http', timeout=12):
        self._host = host
        self._session = session or requests.session()
        self._protocol = protocol
        self._timeout = timeout

    def __repr__(self):
        return f'{self.__class__.__name__}({self._host.__repr__()})'

    def __enter__(self):
        return self

    def __exit__(self, exception_type, exception_value, traceback):
        self.close()

    @property
    def host(self):
        return self._host

    @property
    def session(self):
        return self._session

    @property
    def protocol(self):
        return self._protocol

    @property
    def timeout(self):
        return self._timeout

    @property
    def url_root(self):
        url_root = f'{self._protocol}://{self._host}'
        return url_root

    def login(self, username, password, path='/logincheck'):
        url = self.url_root + path
        data = {
            'username': username,
            'secretkey': password,
        }
        encoded_data = urlencode(data)
        response = self._session.post(
            url=url,
            data=encoded_data,
            timeout=self._timeout
        )
        return response

    def get(self, path, params={}):
        url = self.url_root + path
        response = self._session.get(
            url=url,
            params=params,
            timeout=self.timeout,
        )
        return response

    def post(self, path, data={}, json=''):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.post(
            url=url,
            headers=headers,
            data=data,
            json=json,
            timeout=self.timeout,
        )
        return response

    def put(self, path, data={}, json=''):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.put(
            url=url,
            headers=headers,
            data=data,
            json=json,
            timeout=self.timeout,
        )
        return response

    def delete(self, path):
        url = self.url_root + path
        headers = {
            'X-CSRFTOKEN': self._session.cookies['ccsrftoken'][1:-1],
        }
        response = self._session.delete(
            url=url,
            headers=headers,
            timeout=self.timeout,
        )
        return response

    def logout(self, path='/logout'):
        url = self.url_root + path
        response = self._session.post(
            url=url,
            timeout=self._timeout
        )
        return response

    def close(self):
        self.logout()
        self._session.close()


In [3]:
CREDENTIALS = {
    'host': '150.117.123.248',
    'users': {
        'admin': {
            'username': 'admin',
            'password': '4fcb3244-e5d2-449c-a49d-7b6fa32bfa7f'
        },
        'readonlyadmin': {
            'username': 'readonlyadmin',
            'password': 'cb204a81-0a16-46e9-aaca-2a8cc070593b'
        }
    }
}

In [4]:
import json
class Main():
    
    def __init__(self):
        self.client = FortiAPIClient(CREDENTIALS['host'])

    def login(self):
        self.client.login(
            username=CREDENTIALS['users']['admin']['username'],
            password=CREDENTIALS['users']['admin']['password']
        )
    
    def test_post_firewall_address_with_json_comment_and_check_consistency(self):
        name = '_address__10.65.61.168/32'
        type_ = 'ipmask'
        subnet = '10.65.61.168/32'
        created_by = 'jimmy_lin'
        remark = 'This is a remark field.'
        self.client.post(
            path='/api/v2/cmdb/firewall/address',
            json={
                'name': name,
                'type': type_,
                'subnet': subnet,
                'comment': json.dumps({
                    'created_by': created_by,
                    'remark': remark,
                }),
            }
        )
        r = self.client.get(
            path='/api/v2/cmdb/firewall/address' + '/' + quote('_address__10.65.61.168/32', safe=''),
        )
        r.json()['results'][0]['name']
        r.json()['results'][0]['type']
        r.json()['results'][0]['subnet']
        json.loads(r.json()['results'][0]['comment'])['created_by']
        json.loads(r.json()['results'][0]['comment'])['remark']
        return r
    
    def get_firewall_address(self):
        r = self.client.get(
            path='/api/v2/cmdb/firewall/address',
        )
        return r

    def get_firewall_service(self):
        r = self.client.get(
            path='/api/v2/cmdb/firewall.service/custom/',
            params={
                'fortigate': 'FGVMEV_B6WKV3L0D'
            }
        )
        return r
    
    def get_firewall_ippool(self):
        r = self.client.get(
            path='/api/v2/cmdb/firewall/ippool',
            params={
                'datasource': True
            }
        )
        return r

    def get_firewall_vip(self):
        r = self.client.get(
            path='/api/v2/cmdb/firewall/vip',
        )
        return r

    def get_firewall_policy(self):
        r = self.client.get(
            path='/api/v2/cmdb/firewall/policy',
        )
        return r

    def get_firewall_vip_default(self):
        r = self.client.get(
            path='/api/v2/cmdb/firewall/vip',
            params={
                'action': 'default'
            }
        )
        return r
    
main = Main()
main.login()

In [59]:
client = FortiAPIClient(CREDENTIALS['host'])
client.login(
    username=CREDENTIALS['users']['admin']['username'],
    password=CREDENTIALS['users']['admin']['password']
)

<Response [200]>

In [60]:
r = client.get(
    path='/api/v2/cmdb/system/interface'
)

In [61]:
r.json()['results'][1]

{'name': '_vlan__169',
 'q_origin_key': '_vlan__169',
 'vdom': 'root',
 'vrf': 0,
 'cli-conn-status': 0,
 'fortilink': 'disable',
 'mode': 'static',
 'client-options': [],
 'distance': 5,
 'priority': 0,
 'dhcp-relay-interface-select-method': 'auto',
 'dhcp-relay-interface': '',
 'dhcp-relay-service': 'disable',
 'dhcp-relay-ip': '',
 'dhcp-relay-type': 'regular',
 'dhcp-relay-agent-option': 'enable',
 'management-ip': '0.0.0.0 0.0.0.0',
 'ip': '10.65.62.253 255.255.255.0',
 'allowaccess': '',
 'gwdetect': 'disable',
 'ping-serv-status': 0,
 'detectserver': '',
 'detectprotocol': 'ping',
 'ha-priority': 1,
 'fail-detect': 'disable',
 'fail-detect-option': 'link-down',
 'fail-alert-method': 'link-down',
 'fail-action-on-extender': 'soft-restart',
 'fail-alert-interfaces': [],
 'dhcp-client-identifier': '',
 'dhcp-renew-time': 0,
 'ipunnumbered': '0.0.0.0',
 'username': '',
 'pppoe-unnumbered-negotiate': 'enable',
 'password': '',
 'idle-timeout': 0,
 'detected-peer-mtu': 0,
 'disc-retry

In [51]:
data = {
    'name': '_intf__vlan__1688',
    'vdom': 'root',
    'ip': '10.65.61.253 255.255.255.0',
    'type': 'vlan',
    'vlanid': '168',
    'interface': 'port3',
}
r = client.post(
    path='/api/v2/cmdb/system/interface',
    json=data
)

In [52]:
print(r.text)

{
  "http_method":"POST",
  "revision":"fd6815a8e750cbd01765670f7f87ce3f",
  "revision_changed":true,
  "old_revision":"f0aea18186166391c1099fb40f916d66",
  "mkey":"_intf__vlan__1688",
  "status":"success",
  "http_status":200,
  "vdom":"root",
  "path":"system",
  "name":"interface",
  "serial":"FGVMEV_B6WKV3L0D",
  "version":"v6.4.2",
  "build":1723
}


In [47]:
r = client.get(
    path='/api/v2/cmdb/system/interface',
)

In [48]:
r.json()['results']

[{'name': '_intf__vlan__16',
  'q_origin_key': '_intf__vlan__16',
  'vdom': 'root',
  'vrf': 0,
  'cli-conn-status': 0,
  'fortilink': 'disable',
  'mode': 'static',
  'client-options': [],
  'distance': 5,
  'priority': 0,
  'dhcp-relay-interface-select-method': 'auto',
  'dhcp-relay-interface': '',
  'dhcp-relay-service': 'disable',
  'dhcp-relay-ip': '',
  'dhcp-relay-type': 'regular',
  'dhcp-relay-agent-option': 'enable',
  'management-ip': '0.0.0.0 0.0.0.0',
  'ip': '10.65.61.253 255.255.255.0',
  'allowaccess': '',
  'gwdetect': 'disable',
  'ping-serv-status': 0,
  'detectserver': '',
  'detectprotocol': 'ping',
  'ha-priority': 1,
  'fail-detect': 'disable',
  'fail-detect-option': 'link-down',
  'fail-alert-method': 'link-down',
  'fail-action-on-extender': 'soft-restart',
  'fail-alert-interfaces': [],
  'dhcp-client-identifier': '',
  'dhcp-renew-time': 0,
  'ipunnumbered': '0.0.0.0',
  'username': '',
  'pppoe-unnumbered-negotiate': 'enable',
  'password': '',
  'idle-time

In [36]:
x = print(123)

123
